# Gathering Data and Preprocessing

### 1. Gathering Data

In [1]:
import sys
sys.path.append('/home/mca/Opinion-Mining-Project/src')
from testdata import get_data

df = get_data()
df



,text,label
0,I LOOOVE this product 😍😍!!! Highly recommended...,positive
1,Worst. Experience. Ever. Will NEVER buy again!...,negative
2,"meh... it was okay, I guess. kinda boring tho 🙄",neutral
3,ABSOLUTELY AMAZING SERVICE!!! 😍💯 www.company.com,positive
4,@brand You guys rock! Keep it up 👏🔥🔥🔥,positive
5,"Totally disappointed. Delivery late, product b...",negative
6,just okay. nothing special. 3/10 maybe 🤷‍♂️,neutral
7,"Loved the color, but the fit was terrible :(",negative
8,Refunded. Not worth the price!!! http://badsho...,negative
9,Thanks @brand for the quick support!!,positive


### 2. Change to a csv File

In [2]:
df.to_csv('../../data/test_sample.csv')
df.head()

,text,label
0,I LOOOVE this product 😍😍!!! Highly recommended...,positive
1,Worst. Experience. Ever. Will NEVER buy again!...,negative
2,"meh... it was okay, I guess. kinda boring tho 🙄",neutral
3,ABSOLUTELY AMAZING SERVICE!!! 😍💯 www.company.com,positive
4,@brand You guys rock! Keep it up 👏🔥🔥🔥,positive


### 3. Lowercasing

In [3]:
df['text'] 
print(type(df['text']))  
df['clean_text'] = df['text'].apply(lambda word:word.lower())
df[['text', 'clean_text']]


<class 'pandas.core.series.Series'>


,text,clean_text
0,I LOOOVE this product 😍😍!!! Highly recommended...,i looove this product 😍😍!!! highly recommended...
1,Worst. Experience. Ever. Will NEVER buy again!...,worst. experience. ever. will never buy again!...
2,"meh... it was okay, I guess. kinda boring tho 🙄","meh... it was okay, i guess. kinda boring tho 🙄"
3,ABSOLUTELY AMAZING SERVICE!!! 😍💯 www.company.com,absolutely amazing service!!! 😍💯 www.company.com
4,@brand You guys rock! Keep it up 👏🔥🔥🔥,@brand you guys rock! keep it up 👏🔥🔥🔥
5,"Totally disappointed. Delivery late, product b...","totally disappointed. delivery late, product b..."
6,just okay. nothing special. 3/10 maybe 🤷‍♂️,just okay. nothing special. 3/10 maybe 🤷‍♂️
7,"Loved the color, but the fit was terrible :(","loved the color, but the fit was terrible :("
8,Refunded. Not worth the price!!! http://badsho...,refunded. not worth the price!!! http://badsho...
9,Thanks @brand for the quick support!!,thanks @brand for the quick support!!


###  4. Remove Punctuation and Symbols

In [4]:
import string
print(string.punctuation)
mytable = str.maketrans('','',string.punctuation)
df['clean_text'] = df['clean_text'].apply(lambda word : word.translate(mytable)) 
df['clean_text']


!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


0    i looove this product 😍😍 highly recommended aw...
1        worst experience ever will never buy again 🤮🤬
2           meh it was okay i guess kinda boring tho 🙄
3          absolutely amazing service 😍💯 wwwcompanycom
4                  brand you guys rock keep it up 👏🔥🔥🔥
5    totally disappointed delivery late product bro...
6             just okay nothing special 310 maybe 🤷‍♂️
7            loved the color but the fit was terrible 
8          refunded not worth the price httpbadshopcom
9                   thanks brand for the quick support
Name: clean_text, dtype: object

### 5. Remove Stopwords

In [5]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
print(stopwords.words('english'))
stop_words = set(stopwords.words('english'))
df['no_stopwords'] = df['clean_text'].apply(
  lambda text :  ''.join(word for word in text.split() if word not in stop_words)
)
df

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

[nltk_data] Downloading package stopwords to /home/mca/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,label,clean_text,no_stopwords
0,I LOOOVE this product 😍😍!!! Highly recommended...,positive,i looove this product 😍😍 highly recommended aw...,loooveproduct😍😍highlyrecommendedawesome
1,Worst. Experience. Ever. Will NEVER buy again!...,negative,worst experience ever will never buy again 🤮🤬,worstexperienceeverneverbuy🤮🤬
2,"meh... it was okay, I guess. kinda boring tho 🙄",neutral,meh it was okay i guess kinda boring tho 🙄,mehokayguesskindaboringtho🙄
3,ABSOLUTELY AMAZING SERVICE!!! 😍💯 www.company.com,positive,absolutely amazing service 😍💯 wwwcompanycom,absolutelyamazingservice😍💯wwwcompanycom
4,@brand You guys rock! Keep it up 👏🔥🔥🔥,positive,brand you guys rock keep it up 👏🔥🔥🔥,brandguysrockkeep👏🔥🔥🔥
5,"Totally disappointed. Delivery late, product b...",negative,totally disappointed delivery late product bro...,totallydisappointeddeliverylateproductbroken😡
6,just okay. nothing special. 3/10 maybe 🤷‍♂️,neutral,just okay nothing special 310 maybe 🤷‍♂️,okaynothingspecial310maybe🤷‍♂️
7,"Loved the color, but the fit was terrible :(",negative,loved the color but the fit was terrible,lovedcolorfitterrible
8,Refunded. Not worth the price!!! http://badsho...,negative,refunded not worth the price httpbadshopcom,refundedworthpricehttpbadshopcom
9,Thanks @brand for the quick support!!,positive,thanks brand for the quick support,thanksbrandquicksupport


### 6. Remove Emojis

In [6]:
import emoji

df['no_emoji'] = df['no_stopwords'].apply(lambda row: emoji.replace_emoji(row, replace=''))
df



,text,label,clean_text,no_stopwords,no_emoji
0,I LOOOVE this product 😍😍!!! Highly recommended...,positive,i looove this product 😍😍 highly recommended aw...,loooveproduct😍😍highlyrecommendedawesome,loooveproducthighlyrecommendedawesome
1,Worst. Experience. Ever. Will NEVER buy again!...,negative,worst experience ever will never buy again 🤮🤬,worstexperienceeverneverbuy🤮🤬,worstexperienceeverneverbuy
2,"meh... it was okay, I guess. kinda boring tho 🙄",neutral,meh it was okay i guess kinda boring tho 🙄,mehokayguesskindaboringtho🙄,mehokayguesskindaboringtho
3,ABSOLUTELY AMAZING SERVICE!!! 😍💯 www.company.com,positive,absolutely amazing service 😍💯 wwwcompanycom,absolutelyamazingservice😍💯wwwcompanycom,absolutelyamazingservicewwwcompanycom
4,@brand You guys rock! Keep it up 👏🔥🔥🔥,positive,brand you guys rock keep it up 👏🔥🔥🔥,brandguysrockkeep👏🔥🔥🔥,brandguysrockkeep
5,"Totally disappointed. Delivery late, product b...",negative,totally disappointed delivery late product bro...,totallydisappointeddeliverylateproductbroken😡,totallydisappointeddeliverylateproductbroken
6,just okay. nothing special. 3/10 maybe 🤷‍♂️,neutral,just okay nothing special 310 maybe 🤷‍♂️,okaynothingspecial310maybe🤷‍♂️,okaynothingspecial310maybe
7,"Loved the color, but the fit was terrible :(",negative,loved the color but the fit was terrible,lovedcolorfitterrible,lovedcolorfitterrible
8,Refunded. Not worth the price!!! http://badsho...,negative,refunded not worth the price httpbadshopcom,refundedworthpricehttpbadshopcom,refundedworthpricehttpbadshopcom
9,Thanks @brand for the quick support!!,positive,thanks brand for the quick support,thanksbrandquicksupport,thanksbrandquicksupport


### 7. Tokenization

In [7]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from wordsegment import load, segment

# Download 'punkt' to a custom directory and append it to nltk path
nltk.download('punkt')

# Initialize wordsegment
load()
# Check for NaN or empty strings
df['no_emoji'] = df['no_emoji'].fillna('').astype(str)
print(df['no_emoji'].head())  # Confirm it prints strings like 'absolutelyamazingservice...'

df['segmented'] = df['no_emoji'].apply(lambda text: ' '.join(segment(text)))
tokenizer = lambda text: re.findall(r'\b\w+\b', text.lower())
df['tokenized'] = df['segmented'].apply(tokenizer)
df


[nltk_data] Downloading package punkt to /home/mca/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


0    loooveproducthighlyrecommendedawesome
1              worstexperienceeverneverbuy
2               mehokayguesskindaboringtho
3    absolutelyamazingservicewwwcompanycom
4                        brandguysrockkeep
Name: no_emoji, dtype: object


,text,label,clean_text,no_stopwords,no_emoji,segmented,tokenized
0,I LOOOVE this product 😍😍!!! Highly recommended...,positive,i looove this product 😍😍 highly recommended aw...,loooveproduct😍😍highlyrecommendedawesome,loooveproducthighlyrecommendedawesome,looove product highly recommended awesome,"[looove, product, highly, recommended, awesome]"
1,Worst. Experience. Ever. Will NEVER buy again!...,negative,worst experience ever will never buy again 🤮🤬,worstexperienceeverneverbuy🤮🤬,worstexperienceeverneverbuy,worst experience ever never buy,"[worst, experience, ever, never, buy]"
2,"meh... it was okay, I guess. kinda boring tho 🙄",neutral,meh it was okay i guess kinda boring tho 🙄,mehokayguesskindaboringtho🙄,mehokayguesskindaboringtho,meh okay guess kinda boring tho,"[meh, okay, guess, kinda, boring, tho]"
3,ABSOLUTELY AMAZING SERVICE!!! 😍💯 www.company.com,positive,absolutely amazing service 😍💯 wwwcompanycom,absolutelyamazingservice😍💯wwwcompanycom,absolutelyamazingservicewwwcompanycom,absolutely amazing service www company com,"[absolutely, amazing, service, www, company, com]"
4,@brand You guys rock! Keep it up 👏🔥🔥🔥,positive,brand you guys rock keep it up 👏🔥🔥🔥,brandguysrockkeep👏🔥🔥🔥,brandguysrockkeep,brand guys rock keep,"[brand, guys, rock, keep]"
5,"Totally disappointed. Delivery late, product b...",negative,totally disappointed delivery late product bro...,totallydisappointeddeliverylateproductbroken😡,totallydisappointeddeliverylateproductbroken,totally disappointed delivery late product broken,"[totally, disappointed, delivery, late, produc..."
6,just okay. nothing special. 3/10 maybe 🤷‍♂️,neutral,just okay nothing special 310 maybe 🤷‍♂️,okaynothingspecial310maybe🤷‍♂️,okaynothingspecial310maybe,okay nothing special 310 may be,"[okay, nothing, special, 310, may, be]"
7,"Loved the color, but the fit was terrible :(",negative,loved the color but the fit was terrible,lovedcolorfitterrible,lovedcolorfitterrible,loved color fit terrible,"[loved, color, fit, terrible]"
8,Refunded. Not worth the price!!! http://badsho...,negative,refunded not worth the price httpbadshopcom,refundedworthpricehttpbadshopcom,refundedworthpricehttpbadshopcom,refunded worth price http bad shop com,"[refunded, worth, price, http, bad, shop, com]"
9,Thanks @brand for the quick support!!,positive,thanks brand for the quick support,thanksbrandquicksupport,thanksbrandquicksupport,thanks brand quick support,"[thanks, brand, quick, support]"


### 8. Lemmatization Function

In [8]:
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
import nltk
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')
nltk.download('punkt_tab')
def get_wordnet_pos(treebank_tag):
    # Map POS tag to WordNet POS tag for lemmatizer
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # default
def lemmatize_tokens(token_list):
    #Takes a list of words (tokens) from your sentence.
    #Assigns a Part-of-Speech (POS) tag to each word, like noun, verb, adjective, etc.
    #Returns a list of tuples: (word, POS_tag) for every token.
    pos_tags = pos_tag(token_list)
    print("POS tag : ",pos_tags)
    lemmatizer = WordNetLemmatizer()
    print(lemmatizer)
    lemmatized_sentence = []
    for token, tag in pos_tags:
        wn_tag = get_wordnet_pos(tag)
        lemma = lemmatizer.lemmatize(token, wn_tag)
        lemmatized_sentence.append(lemma)
    
    print("Original:", token_list)
    print("Lemmatized:", " ".join(lemmatized_sentence))
    # "bats" → "bat"

    # "are" → "be"

    # "hanging" → "hang"

    # "feet" → "foot"    
    return lemmatized_sentence


[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/mca/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/mca/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/mca/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### 9. Lemmatization

In [9]:
import pandas as pd

# Apply lemmatization
df['lemmatized'] = df['tokenized'].apply(lemmatize_tokens)
df['lemmatized'] = df['lemmatized'].apply(lambda tokens: " ".join(tokens))
df['lemmatized'] 
df


POS tag :  [('looove', 'JJ'), ('product', 'NN'), ('highly', 'RB'), ('recommended', 'VBD'), ('awesome', 'NN')]
<WordNetLemmatizer>
Original: ['looove', 'product', 'highly', 'recommended', 'awesome']
Lemmatized: looove product highly recommend awesome
POS tag :  [('worst', 'JJS'), ('experience', 'NN'), ('ever', 'RB'), ('never', 'RB'), ('buy', 'VB')]
<WordNetLemmatizer>
Original: ['worst', 'experience', 'ever', 'never', 'buy']
Lemmatized: bad experience ever never buy
POS tag :  [('meh', 'NN'), ('okay', 'NN'), ('guess', 'NN'), ('kinda', 'VBD'), ('boring', 'JJ'), ('tho', 'NN')]
<WordNetLemmatizer>
Original: ['meh', 'okay', 'guess', 'kinda', 'boring', 'tho']
Lemmatized: meh okay guess kinda boring tho
POS tag :  [('absolutely', 'RB'), ('amazing', 'JJ'), ('service', 'NN'), ('www', 'VBD'), ('company', 'NN'), ('com', 'NN')]
<WordNetLemmatizer>
Original: ['absolutely', 'amazing', 'service', 'www', 'company', 'com']
Lemmatized: absolutely amazing service www company com
POS tag :  [('brand', 'NN

,text,label,clean_text,no_stopwords,no_emoji,segmented,tokenized,lemmatized
0,I LOOOVE this product 😍😍!!! Highly recommended...,positive,i looove this product 😍😍 highly recommended aw...,loooveproduct😍😍highlyrecommendedawesome,loooveproducthighlyrecommendedawesome,looove product highly recommended awesome,"[looove, product, highly, recommended, awesome]",looove product highly recommend awesome
1,Worst. Experience. Ever. Will NEVER buy again!...,negative,worst experience ever will never buy again 🤮🤬,worstexperienceeverneverbuy🤮🤬,worstexperienceeverneverbuy,worst experience ever never buy,"[worst, experience, ever, never, buy]",bad experience ever never buy
2,"meh... it was okay, I guess. kinda boring tho 🙄",neutral,meh it was okay i guess kinda boring tho 🙄,mehokayguesskindaboringtho🙄,mehokayguesskindaboringtho,meh okay guess kinda boring tho,"[meh, okay, guess, kinda, boring, tho]",meh okay guess kinda boring tho
3,ABSOLUTELY AMAZING SERVICE!!! 😍💯 www.company.com,positive,absolutely amazing service 😍💯 wwwcompanycom,absolutelyamazingservice😍💯wwwcompanycom,absolutelyamazingservicewwwcompanycom,absolutely amazing service www company com,"[absolutely, amazing, service, www, company, com]",absolutely amazing service www company com
4,@brand You guys rock! Keep it up 👏🔥🔥🔥,positive,brand you guys rock keep it up 👏🔥🔥🔥,brandguysrockkeep👏🔥🔥🔥,brandguysrockkeep,brand guys rock keep,"[brand, guys, rock, keep]",brand guy rock keep
5,"Totally disappointed. Delivery late, product b...",negative,totally disappointed delivery late product bro...,totallydisappointeddeliverylateproductbroken😡,totallydisappointeddeliverylateproductbroken,totally disappointed delivery late product broken,"[totally, disappointed, delivery, late, produc...",totally disappointed delivery late product broken
6,just okay. nothing special. 3/10 maybe 🤷‍♂️,neutral,just okay nothing special 310 maybe 🤷‍♂️,okaynothingspecial310maybe🤷‍♂️,okaynothingspecial310maybe,okay nothing special 310 may be,"[okay, nothing, special, 310, may, be]",okay nothing special 310 may be
7,"Loved the color, but the fit was terrible :(",negative,loved the color but the fit was terrible,lovedcolorfitterrible,lovedcolorfitterrible,loved color fit terrible,"[loved, color, fit, terrible]",love color fit terrible
8,Refunded. Not worth the price!!! http://badsho...,negative,refunded not worth the price httpbadshopcom,refundedworthpricehttpbadshopcom,refundedworthpricehttpbadshopcom,refunded worth price http bad shop com,"[refunded, worth, price, http, bad, shop, com]",refund worth price http bad shop com
9,Thanks @brand for the quick support!!,positive,thanks brand for the quick support,thanksbrandquicksupport,thanksbrandquicksupport,thanks brand quick support,"[thanks, brand, quick, support]",thanks brand quick support


### Tokenization Function   

In [10]:
import re
from wordsegment import load, segment

# Load wordsegment once
load()

def tokenize_sentence(sentence):
    # Step 1: Segment merged words (e.g., "amazingsupport" → "amazing support")
    segmented = ' '.join(segment(sentence))
    
    # Step 2: Tokenize (split into lowercase words)
    tokens = re.findall(r'\b\w+\b', segmented.lower())
    
    return tokens
text = "I recently watched the new action movie and honestly, it was thrilling from start to finish. The plot kept me on the edge of my seat, and the acting was top-notch. However, I felt the soundtrack didn’t quite match the intensity of the scenes. The movie was released in May 2023 and runs for approximately 2 hours and 15 minutes. It grossed over $300 million worldwide in its first week. Many critics praised the director for his bold choices, while some audiences thought the pacing was too fast. Personally, I think the cinematography was stunning, especially in the desert scenes. According to IMDb, the film has a rating of 7.8 out of 10.The lead actor previously starred in a hit sci-fi franchise.In my opinion, this is his best performance yet."
tokenize_sentence(text)

['i',
 'recently',
 'watched',
 'the',
 'new',
 'action',
 'movie',
 'and',
 'honestly',
 'it',
 'was',
 'thrilling',
 'from',
 'start',
 'to',
 'finish',
 'the',
 'plot',
 'kept',
 'me',
 'on',
 'the',
 'edge',
 'of',
 'my',
 'seat',
 'and',
 'the',
 'acting',
 'was',
 'top',
 'notch',
 'however',
 'i',
 'felt',
 'the',
 'soundtrack',
 'didnt',
 'quite',
 'match',
 'the',
 'intensity',
 'of',
 'the',
 'scenes',
 'the',
 'movie',
 'was',
 'released',
 'in',
 'may2023',
 'and',
 'runs',
 'for',
 'approximately',
 '2hoursand15',
 'minutes',
 'it',
 'grossed',
 'over',
 '300',
 'million',
 'worldwide',
 'in',
 'its',
 'first',
 'week',
 'many',
 'critics',
 'praised',
 'the',
 'director',
 'for',
 'his',
 'bold',
 'choices',
 'while',
 'some',
 'audiences',
 'thought',
 'the',
 'pacing',
 'was',
 'too',
 'fast',
 'personally',
 'i',
 'think',
 'the',
 'cinematography',
 'was',
 'stunning',
 'especially',
 'in',
 'the',
 'desert',
 'scenes',
 'according',
 'to',
 'imdb',
 'the',
 'film',
 